In [1]:
class A:
    pass

In [2]:
print(A.__name__)# 类，函数，方法的名字
print(A.__module__)# 当前运行模块名
print(A.__class__)# 对象或类所属的类

A
__main__
<class 'type'>


In [3]:
class A:pass
class B(A):pass
class C(B):pass

In [4]:
print(C.__bases__)# 类的基类的元组，直接父类
print(C.__mro__)# 类的继承解析顺序

(<class '__main__.B'>,)
(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


In [5]:
class D:pass
print(D.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'D' objects>, '__weakref__': <attribute '__weakref__' of 'D' objects>, '__doc__': None}


In [6]:
print(dir(D))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


魔术方法

# hash方法

In [7]:
# 创建初始化和销毁
# 创建 __new__ 创建实例化
# 初始化 __init__
# 销毁 __del__

In [8]:
class AA:
    def __new__(cls,*args, **kwargs):
        print(cls)
        print(args)
        print(kwargs)
        return super().__new__(cls)
    
    def __init__(self, name):
        self.name = name
        print('---------------')
    
a = AA('tom')

<class '__main__.AA'>
('tom',)
{}
---------------


实例化过程中，如果定义了__new__方法，则首先条用__new__方法进行创建实例，并返回本类型实例，然后再调用__init__初始化方法
new方法一定要返回一个cls的实例，否则不会调用init方法，该方法永远都是静态方法
如果没有定义，则调用object方法

In [9]:
class Person:
    def __hash__(self):
        return 1
#         return hash(self.name)
    
    def __eq__(self, other):
        return True
    
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return self.name
tom = Person('tom')
jerry = Person('jerry')

In [10]:
print([tom, jerry])
print({tom, jerry}) # 去重，因为类hash值返回相同，并且eq返回true，则表示重复（去重）
print({tom:1, jerry:2}) # key有去重效果，hash相同，eq相同，则去重（判定后加入的与原有，如果相同，则后加入的去除）。2：赋值效应

[tom, jerry]
{tom}
{tom: 2}


去重：首先比较is是否相同，然后比较eq
hash值相同，不一定去重。还要判断是否相等eq。如果hash值相同，eq内容相等，则会去重

如果要一个类不可hash，则定义__hash__ = None
如果单独定义了一个eq方法，则类自动定义一个__hash__=None

In [11]:
# 练习：设计二维坐标类Point，使其可hash，并比较两个坐标的实例是否相等
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __hash__(self):
#         return hash(self) # 递归调用，错误
        return hash((self.x, self.y))
    
    def __eq__(self, other):
        return self.x == other.x and self.y == self.y
        # return self == other (错误，递归调用，等价==》self.__eq__(other))
        
    def __repr__(self):
        return '{},{}'.format(self.x, self.y)

In [12]:
p1 = Point(1, 3)
p2 = Point(1, 3)
print({p1, p2})

{1,3}


# bool方法

In [13]:
class A: pass
if A:
    print(1, True)

if A():
    print(2, True)

1 True
2 True


当类中没有定义__bool__方法时，默认都是true

In [14]:
class A:
    def __bool__(self):
        return False

if A:
    print(1, '--------------')
if A():
    print(2, '++++++++++++++')

1 --------------


当类中定义了__bool__方法时，则调用bool方法，该方法必须返回bool值，该方法为实例方法，对类没有影响

In [15]:
class A:
    def __len__(self):
        return 1

if A():
    print(1, '************')

1 ************


len返回容器的长度，如果容器为空，等价为false，否则等价为true

In [16]:
class A:
    def __bool__(self):
        return False
    
    def __len__(self):
        return 1

if A():
    print(1, '************')

如果bool和len同时定义了，则优先调用bool，其次len。
如果两者都没有定义，则默认为true

# 可视化

In [17]:
class A:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return 'repr name={}'.format(self.name)
    
    def __str__(self):
        return 'str name={}'.format(self.name)

In [18]:
print(A)
print(A('tom'))
print('{}'.format(A('tom')))
print(str(A('tom')))

<class '__main__.A'>
str name=tom
str name=tom
str name=tom


In [19]:
print(repr(A('tom')))

repr name=tom


如果是str，print，format直接调用，则优先调用str方法，如果没有定义str方法，则调用repr方法，如果没有定义repr方法，则返回对象内存地址信息
除此之外，则调用repr方法，如果repr方法没有定义，则返回对象内存地址信息
一般情况下，直接定义repr方法

运算符重载

In [20]:
class A:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __sub__(self, other):
        return self.age - other.age

    def __isub__(self, other):
#         self.age -= other.age
#         return self
        return self.__class__(self.name, self - other)

In [21]:
tom = A('tom', 20)
jerry = A('jerry', 18)
print(tom - jerry)
tom -= jerry
print(tom.age)

2
2


In [22]:
# 练习：完成point类设计，实现向量的加法
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)# 最好能返回一个同类型
    
    def __iadd__(self, other):
        self.x += other.x
        self.y += other.y
        return self
        
    def __repr__(self):
        return '{},{}'.format(self.x, self.y)

In [23]:
p1 = Point(1, 3)
p2 = Point(2, 4)
p3 = p1 + p2
print(p3)
print(type(p3))
print('-'*30)
p1 += p2
print(p1)

3,7
<class '__main__.Point'>
------------------------------
3,7


In [24]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __gt__(self, other):
        return self.age > other.age
    
    def __ge__(self, other):
        return self.age >= other.age

In [25]:
tom = Person('tom', 20)
jerry = Person('jerry', 30)
print(tom == jerry)
print(tom != jerry)
print(tom > jerry)
print(tom < jerry)
print(tom >= jerry)
print(tom <= jerry)

False
True
False
True
False
True


# 容器方法

__len__:
    内建函数，返回对象的长度（>=0的整数），bool函数调用的时候，如果没有定义bool方法，则调用len方法
__iter__:
    迭代容器，返回一个新的迭代对象
__contains__:
    in成员运算符，调用iter方法遍历
__getitem__:
    实现self[key]时访问。key不存在时引发keyerror异常
__setitem__:
    和getitem类似，设置值得方法
__missint__:
    字典或子类使用getitem时，key不存在时执行该方法

In [26]:
# 练习：将购物车改造成容器类
class Cart:
    def __init__(self):
        self.item = []
    
    def additem(self, item):
        self.item.append(item)
    
    def __len__(self):
        return len(self.item)
    
    def __iter__(self):
#         return iter(self.item)
        yield from self.item
    
    def __getitem__(self, index):
        return self.item[index]
    
    def __setitem__(self, index, value):
        self.item[index] = value
        
    def __add__(self, other):
        self.item.append(other)
        return self

In [27]:
car = Cart()
car.additem(1)
car.additem(2)
car.additem(3)
car[2] = 'abc'
print(len(car))
print(2 in car)
print(12 in car)
car + 5 + 77 + 100
print(list(car))
print(car[2])

3
True
False
[1, 2, 'abc', 5, 77, 100]
abc


# 可调用对象

In [28]:
def fn(): pass
callable(fn)

True

In [29]:
class A:
    def __call__(self, *args, **kwargs):
        print('call')

In [30]:
a = A()
a()

call


在类中定义个call方法，实例就可以像函数一样调用

In [31]:
#练习：定义一个斐波那契数列的类，方便调用，计算第n项
class Fib:
    def __init__(self):
        self.item = [0, 1, 1]
        
    def __call__(self, n):
        if n >= len(self.item):
            for i in range(len(self.item), n+1):
                self.item.append(self.item[i-1] + self.item[i-2])
        return self.item[n]
    
    def __getitem__(self, index):
        return self(index)
    
    def __len__(self):
        return len(self.item)
    
    def __iter__(self):
        yield from self.item

In [32]:
f = Fib()
print(f(10))
print(f[15])
print(f.item)
print(len(f))
for x in f:
    print(x, end=' ')

55
610
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]
16
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 

# 上下文管理

In [33]:
class A:
    def __init__(self):
        print('init')
    
    def __enter__(self):
        print('enter')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit')

In [34]:
a = A()
with a as f:
    print(a == f)# 如果用as语法，其enter的返回值为f的值
    print(a, f)

init
enter
True
<__main__.A object at 0x0000000004E8C400> <__main__.A object at 0x0000000004E8C400>
exit


应用场景：
    1. 增强功能，类似装饰器，在enter和exit功能添加功能
    2. 资源管理，打开资料需要关闭
    3. 权限验证，在enter中处理

In [35]:
from contextlib import contextmanager

In [36]:
@contextmanager
def foo():
    print('enter')
    try:
        yield 123
    finally:
        print('exit')

In [37]:
with foo() as f:
    print('-'*30)
    print(f)
    print('-'*30)

enter
------------------------------
123
------------------------------
exit


用contexemanager为一个函数，增加上下文管理的机制
yield之前作为enter的方式执行
yield之后作为exit的方式执行
yield的值作为enter的返回值

# 反射

In [38]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [39]:
Point.z = 100
setattr(Point, 'z', 500) # 两个方法等价，都是增加类属性
print(Point.__dict__)

{'__module__': '__main__', '__init__': <function Point.__init__ at 0x0000000004EA0488>, '__dict__': <attribute '__dict__' of 'Point' objects>, '__weakref__': <attribute '__weakref__' of 'Point' objects>, '__doc__': None, 'z': 500}


In [40]:
p1 = Point(1, 4)
p1.x, getattr(p1, 'x')# 两个方法等价, 获取实例属性

(1, 1)

In [41]:
Point.show = lambda self:print(self.x, self.y)
# setattr(Point, 'show', lambda self: print(self.x, self.y)) 该方法等价于上一行，增加类方法
p2 = Point(2, 5)
p2.show() # 该方式下增加的方法，会自动绑定self
print(p2.__dict__)
print(Point.__dict__)

2 5
{'x': 2, 'y': 5}
{'__module__': '__main__', '__init__': <function Point.__init__ at 0x0000000004EA0488>, '__dict__': <attribute '__dict__' of 'Point' objects>, '__weakref__': <attribute '__weakref__' of 'Point' objects>, '__doc__': None, 'z': 500, 'show': <function <lambda> at 0x0000000004EA0D08>}


In [42]:
p1.show = lambda self:print(self.x, self.y)
# setattr(p1, 'show', lambda self:print(self.x, self.y))  该方式等价于p1.show。。。增加实例方法
p1.show(p1) # 该方式下定义的实例方法，不会绑定self，调用的时候，需要手动传入
print(p1.__dict__)

1 4
{'x': 1, 'y': 4, 'show': <function <lambda> at 0x0000000004EA0F28>}


反射：运行时，动态的增删类或者实例的方式，装饰器和mixin都是定义时就确定的，因此反射能力具有更大的灵活性

In [43]:
# 练习：命令分发器，通过名称查到对应的函数执行
class Dispatcher: 
    def reg(self, cmd, fn):
        setattr(self, cmd, fn)
    
    def run(self):
        while True:
            cmd = input('>>>').strip()
            if cmd == 'quit':
                break
            para = input('input para >>>').split(',')
            getattr(self, cmd, lambda :print('unknown cmd {}'.format(cmd)))(*para)

In [44]:
def add(x, y):
    print(x + y)
    
dis = Dispatcher()
dis.reg('a', add)
dis.run()

>>>a
input para >>>1,2
12
>>>quit


In [53]:
class A:
    z = 1000
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getattr__(self, item):
        print(item, '-'*30)

In [54]:
a = A(1, 4)
a.x
a.z
a.b

b ------------------------------


一个类的属性会按照继承关系查找，如果找不到，则调用getattr方法，如果没有这个方法，则抛出异常

In [61]:
class A:
    z = 1000
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __setattr__(self, key, value):
        print(key, value, '++++++++++++++')
        self.__dict__[key] = value

In [64]:
a = A(1, 2)
a.x = 5
print(a.__dict__)
print(a.x)

x 1 ++++++++++++++
y 2 ++++++++++++++
x 5 ++++++++++++++
{'x': 5, 'y': 2}
5


实例通过.号设置属性，例如：a.x=10，就会调用setattr，属性要添加到实例额的dict中，这个需要自己完成

In [69]:
class A:
    z = 1000
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __delattr__(self, item):
        print('*'*30)

In [71]:
a = A(1,2)
del a.x

******************************


实例通过del删除实例属性时，则调用该方法

In [72]:
class A:
    z = 1000
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getattribute__(self, item):
        print('*'*30)
        return object.__getattribute__(self, item)

In [74]:
a = A(1, 2)
print(a.x)

******************************
1


实例的所有属性访问，第一个都会调用getattribute方法，他可以阻止属性的查找，

总结：
    getattr 当通过搜索实例，实例的类及祖先类查找不到属性，则会调用此方法
    setattr 通过.访问实例属性，进行增加，修改，都会调用该方法
    delattr 当通过实例来删除属性时，调用该方法
    getattribute 实例所有的属性调用都从这个方法开始

# 描述器

In [92]:
class A:
    def __init__(self):
        print('A(). init~~~~~~~~~~~')
        self.x = 100
        
    def __get__(self, instance, owner):
        print(self, instance, owner, '$'*30)
        return self
    
    def __repr__(self):
        return '<A {}>'.format(self.x)
class B:
    a = A()
    def __init__(self):
        self.y = 200

A(). init~~~~~~~~~~~


In [93]:
# B类的调用
print(B.a)
print(B.a.x)

<A 100> None <class '__main__.B'> $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<A 100>
<A 100> None <class '__main__.B'> $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
100


In [94]:
# b的实例调用
b = B()
print(b.a)
print(b.a.x)

<A 100> <__main__.B object at 0x000000000828FBE0> <class '__main__.B'> $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<A 100>
<A 100> <__main__.B object at 0x000000000828FBE0> <class '__main__.B'> $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
100


In [100]:
class A:
    def __init__(self):
        print('A(). init~~~~~~~~~~~')
        self.x = 100
        
    def __get__(self, instance, owner):
        print(self, instance, owner, '$'*30)
        return self
    
    def __set__(self, instance, value):
        print(self, instance, value, '-'*30)
    
    def __repr__(self):
        return '<A {}>'.format(self.x)
class B:
    a = A()
    def __init__(self):
        self.y = 200

A(). init~~~~~~~~~~~


In [105]:
b = B()
b.y = 100

In [107]:
b.a = 1000

<A 1> <__main__.B object at 0x000000000828F860> 1000 ------------------------------


In [115]:
class A:
    def __init__(self):
        print('A(). init~~~~~~~~~~~')
        self.x = 100
        
    def __get__(self, instance, owner):
        print(self, instance, owner, '$'*30)
        return self
    
    def __set__(self, instance, value):
        print(self, instance, value, '-'*30)
    
    def __delete__(self, instance):
        print(self, instance, '+'*30)
    
    def __repr__(self):
        return '<A {}>'.format(self.x)
class B:
    a = A()
    def __init__(self):
        self.x = 200

A(). init~~~~~~~~~~~


In [116]:
b = B()
del b.a

<A 100> <__main__.B object at 0x000000000828FDA0> ++++++++++++++++++++++++++++++


In [123]:
b.a.x = 222

<A 222> <__main__.B object at 0x000000000828FDA0> <class '__main__.B'> $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [113]:
b = B()
del b.a

<A 100> <__main__.B object at 0x00000000086B88D0> ++++++++++++++++++++++++++++++


总结：
    一个类，实现get，set，delete三个方法中的任何一个，就是描述器
    只实现get方法的，是非数据描述器
    同时实现了get，set方法的就是数据描述器
    
    非数据描述器，属性访问时，优先使用自己的
    数据描述器，属性访问时，优先使用描述器的

In [131]:
# 练习：staticmethod, classmethod方法实现
class StaticMethod:
    def __init__(self, fn):
        self.fn = fn
    
    def __get__(self, instance, owner):
        print(self, instance, owner)
        return self.fn
    
from functools import partial
class ClassMethod:
    def __init__(self, fn):
        self.fn = fn
    
    def __get__(self, instance, owner):
        print(self, instance, owner)
        return partial(self.fn, owner) # 偏函数，先固定函数的一个参数
    
class A:
    @StaticMethod  # foo = StaticMethod(foo)  这就是一个非数据描述器
    def foo():
        print('static method called')

    @ClassMethod # bar = ClassMethod(bar)  这就是一个非数据描述器
    def bar(cls):
        print('class method called')
# 静态类方法调用
a = A()
a.foo()
print('+'*30)
# 类方法调用
a.bar()

<__main__.StaticMethod object at 0x0000000007ED0EB8> <__main__.A object at 0x0000000007ED0F28> <class '__main__.A'>
static method called
++++++++++++++++++++++++++++++
<__main__.ClassMethod object at 0x0000000007ED0FD0> <__main__.A object at 0x0000000007ED0F28> <class '__main__.A'>
class method called


In [192]:
# 练习：数据类型检查     普通做法
class Person:
    def __init__(self, name:str, age:int):
        params = ((name, str), (age, int))
        if not self.checkdata(params):
            raise TypeError
        self.name = name
        self.age = age
        print('ok -----------------')
        
    def checkdata(self, params):
        for k,v in params:
            if not isinstance(k, v):
                return False
        return True

p1 = Person('tom', 18)
p2 = Person('jerry', 1)

ok -----------------
ok -----------------


In [145]:
# 练习：数据类型检查     描述器做法
class TypeCheck:
    def __init__(self, name, type):
        self.name = name
        self.type = type
        
    def __get__(self, instance, owner):
        if instance is not None:
            return instance.__dict__[self.name]
        return self
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError
        instance.__dict__[self.name] = value # 数据保存在Person实例的字典中

class Person:
    # 要用描述器，则必须用类属性。而且数据描述器的优先级高于实例的属性
    name = TypeCheck('name', str)  # 输入的已知：name 和 name的类型
    age = TypeCheck('age', int)  # 输入的已知：age 和 age的类型
    
    def __init__(self, name:str, age:int):
        self.name = name
        self.age = age

p1 = Person('tom', 18)
print(p1.__dict__)
print(p1.name, p1.age)

{'name': 'tom', 'age': 18}
tom 18


In [168]:
# 练习：数据类型检查     描述器做法
class TypeCheck:
    def __init__(self, name, type):
        self.data = {} # 自己开辟一个字典，存储属性
        self.name = name
        self.type = type
        
    def __get__(self, instance, owner):
        if instance is not None:
            return self.data[self.name]
        return self
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError
        self.data[self.name] = value

class Person:
    # 要用描述器，则必须用类属性。而且数据描述器的优先级高于实例的属性
    name = TypeCheck('name', str)  # 输入的已知：name 和 name的类型
    age = TypeCheck('age', int)  # 输入的已知：age 和 age的类型
    
    def __init__(self, name:str, age:int):
        self.name = name
        self.age = age

p1 = Person('tom', 12)
print(p1.name, p1.age)

tom 12


In [184]:
# 练习：数据类型检查     描述器做法 同时使用装饰器
class TypeCheck:
    def __init__(self, name, type):
        self.name = name
        self.type = type
        self.data = {}
    
    def __get__(self, instance, owner):
        if instance is not None:
            return self.data[self.name]
        return self
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError
        self.data[self.name] = value

import inspect
def typeassert(cls):
    sig = inspect.signature(cls)
    params = sig.parameters
    for name, param in params.items():
        if param.annotation != param.empty:
            setattr(cls, name, TypeCheck(name, param.annotation))
    return cls

@typeassert # Person = typeassert(Person)
class Person:
    def __init__(self, name:str, age:int):
        self.name = name
        self.age = age

p1 = Person('tom', 11)
print(p1.name, p1.age)

tom 11


In [191]:
# 练习：数据类型检查     描述器做法 同时使用装饰器, 将上面的装饰函数，改成装饰类
class TypeCheck:
    def __init__(self, name, type):
        self.name = name
        self.type = type
        self.data = {}
    
    def __get__(self, instance, owner):
        if instance is not None:
            return self.data[self.name]
        return self
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError
        self.data[self.name] = value

import inspect
class TypeAssert:
    def __init__(self, cls):
        self.cls = cls
    
    def __call__(self, *args, **kwargs): # 因为TypeAssert要可调用，则用call方法，构建一个新的cls实例，返回出去，给Person
        sig = inspect.signature(self.cls)
        params = sig.parameters
        for name, param in params.items():
            if param.annotation != param.empty:
                setattr(self.cls, name, TypeCheck(name, param.annotation))
        return self.cls(*args, **kwargs)

@TypeAssert # Person = TypeAssert(Person)
class Person:
    def __init__(self, name:str, age:int):
        self.name = name
        self.age = age

p1 = Person('tom', 1)
print(p1.name, p1.age)

tom 1


In [225]:
# 进阶题：property 数据描述器  作用是将原来实例访问方式a.data()变成a.data，类方法的访问方式变成类似变量名的访问方式
class Property:
    def __init__(self, fget, fset=None):
        self.fset = fset
        self.fget = fget
    
    def __get__(self, instance, owner):
        if instance is not None:
            return self.fget(instance) # 因为是实例定义的方法，所以不会自动绑定self，需要手动传入参数
        return self
    
    def __set__(self, instance, value):
        if instance is not None:
            self.fset(instance, value)
    
    def setter(self, fset):
        self.fset = fset
        return self
        
class A:
    def __init__(self, x):
        self._x = x
    
    @Property # data = Property(data)
    def data(self):
        return self._x
    
    @data.setter # 首先@data就会调用Property类（描述器），然后.setter去调用Property类的setter方法，返回本实例。
                #然后再调用Property类的set方法，由set方法再去调用A类的data方法进行赋值
    def data(self, value):
        print('===========')
        self._x = value
    
a = A(10)
print(a.data)
a.data = 100
print(a.data)

10
100


A.show = lambda self:pass
a.show() #上面定义的类属性，用实例调用时，是有绑定效果，会自动传参self
a.show = lambda self:pass
a.show(self) #上面用实例定义的属性，self参数需要手动传入，不会绑定效果